In [1]:
raw_dataset_path = '../datasets/BTCUSDT-Trades/'
output_path = '../output'
output_base_path = '../output'

In [2]:
import os
from loguru import logger
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd

logger.remove()
logger.add(lambda msg: print(msg, end=""), level="INFO")

from utils import create_dollar_bars

df_dask = dd.read_parquet(os.path.join(raw_dataset_path, 'BTCUSDT-Dataset-part-1.parquet'), columns=[])

df_dask.head()

,trade_id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,0,4261.48,0.100000,426.148000,2017-08-17 04:00:28.322,True,True
1,1,4261.48,1.600000,6818.368000,2017-08-17 04:00:32.285,True,True
2,2,4261.48,0.075183,320.390851,2017-08-17 04:00:32.322,False,True
3,3,4280.56,0.029600,126.704576,2017-08-17 04:02:48.879,False,True
4,4,4280.56,0.231474,990.838345,2017-08-17 04:02:48.887,False,True


In [4]:
df_dask['qty'].shift(-1).head()

0    1.600000
1    0.075183
2    0.029600
3    0.231474
4    0.000234
Name: qty, dtype: float64

In [ ]:
df['side'] = np.where(df['price'].shift() > df['price'], 1, np.where(df['price'].shift() < df['price'], -1, np.nan))

In [5]:
import os
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

# Configuração do cluster Dask com limite de memória de 32 GB
cluster = LocalCluster(
    n_workers=4,            # Número de workers; ajuste conforme seu hardware
    threads_per_worker=3,   # Threads por worker
    memory_limit='8GB'      # Memória por worker (4 workers x 8GB = 32GB)
)
client = Client(cluster)

print(client)

# Padrão para ler todos os arquivos Parquet
parquet_pattern = os.path.join(raw_dataset_path, 'BTCUSDT-Dataset-part-*.parquet')

# Leitura dos arquivos Parquet com Dask
df_dask = dd.read_parquet(parquet_pattern, columns=['price', 'qty', 'quoteQty', 'time'])

print(df_dask)

# Encerrar o cliente Dask após a conclusão
client.close()


<Client: 'tcp://127.0.0.1:56878' processes=4 threads=12, memory=29.80 GiB>
Dask DataFrame Structure:
                   price      qty quoteQty            time
npartitions=469                                           
                 float64  float64  float64  datetime64[ns]
                     ...      ...      ...             ...
...                  ...      ...      ...             ...
                     ...      ...      ...             ...
                     ...      ...      ...             ...
Dask Name: read_parquet, 1 expression
Expr=ReadParquetFSSpec(b98da6b)


In [6]:
import os
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing
import datetime

# 1. Configuração do Cluster Dask com ajustes para otimização de memória
def setup_dask_cluster():
    num_cores = multiprocessing.cpu_count()
    print(f"Número de núcleos disponíveis: {num_cores}")

    # Definindo número de workers e threads por worker
    # Ajuste conforme seu sistema e necessidades
    # Por exemplo, 8 núcleos: 2 workers com 4 threads cada
    n_workers = 2  # Reduzido para menos workers
    threads_per_worker = 4  # Mais threads por worker

    cluster = LocalCluster(
        n_workers=n_workers,
        threads_per_worker=threads_per_worker,
        memory_limit='8GB'  # Ajuste a memória por worker conforme necessário
    )

    client = Client(cluster)
    print(client)
    return client

# 2. Leitura dos Arquivos Parquet com otimização de memória
def read_parquet_files(raw_dataset_path):
    parquet_pattern = os.path.join(raw_dataset_path, 'BTCUSDT-Dataset-part-*.parquet')
    # Especifique apenas as colunas necessárias e otimize os tipos de dados se possível
    df_dask = dd.read_parquet(
        parquet_pattern,
        columns=['price', 'qty', 'isBuyerMaker', 'time'],
        engine='pyarrow'  # Utilize o engine 'pyarrow' para melhor performance
    )
    return df_dask

# 3. Aplicação das Operações Matemáticas
def apply_operations(df_dask):
    # Calcular 'trade_dollar' = price * qty
    df_dask['trade_dollar'] = df_dask['price'] * df_dask['qty']

    # Calcular 'dollar_side' = -1 se isBuyerMaker for True, 1 se False
    df_dask['dollar_side'] = df_dask['isBuyerMaker'].map({True: -1, False: 1})

    # Calcular 'dollar_imbalance' = trade_dollar * dollar_side
    df_dask['dollar_imbalance'] = df_dask['trade_dollar'] * df_dask['dollar_side']

    return df_dask

# 4. Salvar o DataFrame Processado
def save_processed_dataframe(df_dask, output_path):
    os.makedirs(output_path, exist_ok=True)
    try:
        df_dask.to_parquet(
            output_path,
            write_index=False,
            compression='snappy',
            partition_size='500MB',  # Reduzido para partições menores
            overwrite=True
        )
        print(f"DataFrame processado salvo com sucesso em: {output_path}")
    except Exception as e:
        print(f"Erro ao salvar o DataFrame: {e}")

# 5. Função Principal para Orquestrar o Processo
def main():


    # Configuração do Cluster
    client = setup_dask_cluster()

    try:
        # Leitura dos Arquivos
        df_dask = read_parquet_files(raw_dataset_path)
        print("Arquivos Parquet lidos com sucesso.")

        # Aplicação das Operações Matemáticas
        df_dask = apply_operations(df_dask)
        print("Operações matemáticas aplicadas com sucesso.")

        # Opcional: Persistir os dados na memória para otimizar múltiplas operações
        # df_dask = df_dask.persist()

        # Cálculo de Métricas
        media_preco = df_dask['price'].mean()
        total_dollar_imbalance = df_dask['dollar_imbalance'].sum()
        df_filtrado = df_dask[df_dask['qty'] > 1000]

        # Executar o processamento com tratamento de erros
        try:
            resultado_media_preco = media_preco.compute()
            resultado_total_dollar_imbalance = total_dollar_imbalance.compute()
            resultado_df_filtrado = df_filtrado.compute()

            print(f"Média do preço: {resultado_media_preco}")
            print(f"Total do dollar imbalance: {resultado_total_dollar_imbalance}")
            print("DataFrame filtrado (exemplo):")
            print(resultado_df_filtrado.head())
        except Exception as compute_error:
            print(f"Erro durante a computação das métricas: {compute_error}")

        # Definir Caminho de Saída com Versionamento
        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        output_path = f'{output_base_path}_v{timestamp}'

        # Salvar o DataFrame Processado
        save_processed_dataframe(df_dask, output_path)

        # Validação dos Dados Salvos
        try:
            df_reloaded = dd.read_parquet(output_path)
            media_preco_reloaded = df_reloaded['price'].mean().compute()
            total_dollar_imbalance_reloaded = df_reloaded['dollar_imbalance'].sum().compute()

            print(f"Média do preço após o salvamento: {media_preco_reloaded}")
            print(f"Total do dollar imbalance após o salvamento: {total_dollar_imbalance_reloaded}")
        except Exception as validation_error:
            print(f"Erro ao validar os dados salvos: {validation_error}")

    finally:
        # Encerrar o Cliente Dask
        client.close()
        print("Cliente Dask encerrado.")

# Executar a Função Principal
if __name__ == "__main__":
    main()


Número de núcleos disponíveis: 14


/opt/anaconda3/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56908 instead
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/dask_expr/_collection.py:4160: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('isBuyerMaker', 'int64'))

  warnings.warn(meta_warning(meta))
2024-12-21 16:52:38,376 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 128, in unpackb
    ret = unpacker._unpack()
          ^^^^^^

<Client: 'tcp://127.0.0.1:56909' processes=2 threads=8, memory=14.90 GiB>
Arquivos Parquet lidos com sucesso.
Operações matemáticas aplicadas com sucesso.
Erro durante a computação das métricas: ('mean_aggregate-639fbc2a14536e703350db49f712e37b', 0)


2024-12-21 16:52:42,114 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 128, in unpackb
    ret = unpacker._unpack()
          ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 565, in _unpack
    ret.append(self._unpack(EX_CONSTRUCT))
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 585, in _unpack
    key = self._unpack(EX_CONSTRUCT)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 546, in _unpack
    typ, n, obj = self._read_header()
                  ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 447, in _read_header
    self._reserve(1)
  File "/opt/anaconda3/lib/python3.12/site-packages/msgpack/fallback.py", line 420, 

Erro ao salvar o DataFrame: ('toparquetbarrier-4c954778084d6eadccce89538bd1241b', 0)
Erro ao validar os dados salvos: No files satisfy the `parquet_file_extension` criteria (files must end with ('.parq', '.parquet', '.pq')).
Cliente Dask encerrado.


In [ ]:
import dask
import distributed
import pkg_resources

print("Dask version:", dask.__version__)
print("Distributed version:", distributed.__version__)

try:
    msgpack_version = pkg_resources.get_distribution("msgpack").version
    print("msgpack version:", msgpack_version)
except Exception as e:
    print(f"Erro ao obter a versão do msgpack: {e}")

In [ ]:
import pandas as pd

# Criando um DataFrame de exemplo
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

# Deletando o DataFrame
del df

In [ ]:
import gc

# Forçando a coleta de lixo
gc.collect()